<a href="https://colab.research.google.com/github/LorenzoBioinfo/DrugDiscoveryML/blob/Dev/VirtualScreening_Alzheimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Virtual Screening Pipeline — BACE1 Inhibitor Discovery

## Project Overview

This notebook implements a **ligand-based virtual screening pipeline** applied to the discovery
of potential inhibitors of **BACE1 (Beta-Secretase 1)**, a key therapeutic target in
**Alzheimer's Disease**.

BACE1 is a transmembrane aspartyl protease responsible for the cleavage of the amyloid
precursor protein (APP), leading to the production of amyloid-β peptides — the main
component of amyloid plaques found in Alzheimer's patients. Inhibiting BACE1 is therefore
one of the most studied strategies to slow or prevent neurodegeneration.

## Workflow

Starting from a large library of purchasable compounds (ZINC20), we apply a sequential
filtering and ranking pipeline:

1. **Library Loading** — import raw compound library in SDF format
2. **Physicochemical Filtering** — Lipinski's Rule of Five to ensure drug-likeness
3. **Structural Alerts Filtering** — PAINS and BRENK filters to remove reactive,
   toxic or otherwise undesirable compounds
4. **Fingerprint Generation** — ECFP6 molecular fingerprints
5. **Similarity Search** — Tanimoto similarity against known BACE1 inhibitors (ChEMBL)
6. **Ranking & Analysis** — ranked list of candidate molecules for further investigation

## Data Sources

- **Screening Library**: ZINC20 Purchasable subset
- **Reference Actives**: BACE1 inhibitors from ChEMBL (IC50 < 1000 nM, target ID: CHEMBL4822)

In [12]:
!pip install numpy>=1.26.4,<2.0.0 scipy>=1.15.0 pandas>=2.2.3 matplotlib>=3.10.0 seaborn>=0.13.2 rdkit>=2024.3.2 scikit-learn>=1.6.0 tqdm>=4.67.1 joblib>=1.4.0 jupyterlab>=4.3.0
!pip install chembl_webresource_client

/bin/bash: line 1: 2.0.0: No such file or directory


In [36]:
# ============================================================
# PART 1 — Download Datasets
# ============================================================
#   LIBRARY   → ChEMBL drug-like compounds
#   REFERENCE → BACE1 inhibitors (GitHub/DeepChem)
# ============================================================

import os
import pandas as pd
from chembl_webresource_client.new_client import new_client

os.makedirs('data', exist_ok=True)

# ============================================================
# LIBRARY — ChEMBL drug-like compounds
# ============================================================
# Filtri applicati direttamente via API:
#   MW ≤ 500, LogP ≤ 5, HBD ≤ 5, HBA ≤ 10
# → già Lipinski-compliant, pronto per il filtering step

print("Downloading drug-like library from ChEMBL...")

molecule = new_client.molecule
compounds = molecule.filter(
    molecule_properties__mw_freebase__lte=500,
    molecule_properties__alogp__lte=5,
    molecule_properties__hbd__lte=5,
    molecule_properties__hba__lte=10,
    molecule_type='Small molecule'
).only([
    'molecule_chembl_id',
    'molecule_structures',
    'molecule_properties'
])

records = []
for comp in compounds:
    smiles = comp.get('molecule_structures') or {}
    props  = comp.get('molecule_properties') or {}
    canonical = smiles.get('canonical_smiles')
    if canonical:
        records.append({
            'SMILES'    : canonical,
            'CHEMBL_ID' : comp.get('molecule_chembl_id'),
            'MW'        : props.get('mw_freebase'),
            'LogP'      : props.get('alogp'),
            'HBD'       : props.get('hbd'),
            'HBA'       : props.get('hba'),
        })

df_library = pd.DataFrame(records)
df_library.to_csv('data/chembl_library.csv', index=False)
print(f"Library size: {len(df_library)} molecules")

# ============================================================
# REFERENCE SET — BACE1 Inhibitors (Alzheimer)
# ============================================================

print("\nDownloading BACE1 reference set...")
url_bace = "https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/bace.csv"
df_ref = pd.read_csv(url_bace)
df_ref = df_ref.rename(columns={'mol': 'SMILES', 'Class': 'active'})

df_ref_actives = df_ref[df_ref['active'] == 1].copy()
df_ref_actives.to_csv('data/bace1_actives.csv', index=False)

# ============================================================
# Quick check
# ============================================================
print("\n--- Summary ---")
print(f"Library          : {len(df_library)} drug-like molecules (ChEMBL)")
print(f"Reference actives: {len(df_ref_actives)} BACE1 inhibitors")
print(f"\nLibrary columns  : {df_library.columns.tolist()}")
print(f"Reference columns: {df_ref_actives.columns.tolist()}")
print("\nLibrary preview:")
print(df_library.head(3))
print("\nReference preview:")
print(df_ref_actives.head(3))

KeyboardInterrupt: 

## Utils functions



In [23]:
import os
import pickle
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from pathlib import Path
def save_molecular_dataframe(df, filename, chapter="ch01", compress=True):
    """
    Save a pandas DataFrame containing molecular data to a pickle file.

    This function handles dataframes that contain mixed data types including
    RDKit Mol objects, which cannot be saved with standard methods like
    df.to_csv() or df.to_parquet().

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame to save, which may contain RDKit Mol objects
    filename : str
        Name of the file (without path)
    chapter : str
        Chapter identifier for directory organization
    compress : bool
        Whether to use compression (recommended for large dataframes)

    Returns:
    --------
    str
        Path to the saved file
    """
    # Create the artifacts directory if it doesn't exist
    save_dir = Path(f"artifacts/{chapter}/")
    save_dir.mkdir(parents=True, exist_ok=True)

    # Add .pkl or .pkl.gz extension if not present
    if not filename.endswith('.pkl') and not filename.endswith('.pkl.gz'):
        filename = f"{filename}.pkl"

    # Add compression extension if requested
    if compress and not filename.endswith('.gz'):
        filename = f"{filename}.gz"

    # Full path for saving
    save_path = save_dir / filename

    # Save the dataframe using pickle with optional compression
    protocol = pickle.HIGHEST_PROTOCOL  # Use the most efficient protocol

    print(f"Saving dataframe with {len(df)} rows to {save_path}...")
    with open(save_path, 'wb') as f:
        pickle.dump(df, f, protocol=protocol)

    file_size_mb = os.path.getsize(save_path) / (1024 * 1024)
    print(f"Successfully saved dataframe ({file_size_mb:.1f} MB)")

    return str(save_path)

def load_molecular_dataframe(filename, chapter="ch01"):
    """
    Load a pandas DataFrame containing molecular data from a pickle file.

    Parameters:
    -----------
    filename : str
        Name of the file to load (without path)
    chapter : str
        Chapter identifier for directory organization

    Returns:
    --------
    pandas.DataFrame
        The loaded DataFrame
    """
    # Create the full file path
    file_dir = Path(f"artifacts/{chapter}/")

    # Handle different possible file extensions
    if not (filename.endswith('.pkl') or filename.endswith('.pkl.gz')):
        # Try both compressed and uncompressed versions
        if (file_dir / f"{filename}.pkl.gz").exists():
            filename = f"{filename}.pkl.gz"
        else:
            filename = f"{filename}.pkl"

    file_path = file_dir / filename

    # Check if file exists
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    print(f"Loading molecular dataframe from {file_path}...")
    start_time = pd.Timestamp.now()

    # Load the dataframe
    with open(file_path, 'rb') as f:
        df = pickle.load(f)

    # Calculate loading time
    load_time = (pd.Timestamp.now() - start_time).total_seconds()

    print(f"Successfully loaded dataframe with {len(df)} rows and {len(df.columns)} columns")
    print(f"Loading time: {load_time:.2f} seconds")

    return df

def list_saved_dataframes(chapter="ch01"):
    """
    List all saved dataframes in the artifacts directory.

    Parameters:
    -----------
    chapter : str
        Chapter identifier for directory organization

    Returns:
    --------
    list
        List of available dataframe filenames
    """
    save_dir = Path(f"artifacts/{chapter}/")

    if not save_dir.exists():
        print(f"No artifacts directory found for chapter {chapter}")
        return []

    # Get all pickle files
    saved_files = list(save_dir.glob("*.pkl*"))

    if not saved_files:
        print(f"No saved dataframes found in {save_dir}")
        return []

    # Print information about available files
    print(f"Available saved dataframes in {save_dir}:")
    file_info = []

    for file_path in saved_files:
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
        modified_time = pd.Timestamp.fromtimestamp(os.path.getmtime(file_path))

        file_info.append({
            'filename': file_path.name,
            'size_mb': file_size_mb,
            'modified': modified_time
        })

    # Sort by modification time (newest first)
    file_info.sort(key=lambda x: x['modified'], reverse=True)

    # Display the information
    for info in file_info:
        print(f"  {info['filename']} ({info['size_mb']:.1f} MB, modified: {info['modified']})")

    return [info['filename'] for info in file_info]



# Matplotlib and Seaborn setup for consistent visualizations
def setup_visualization_style():
    """Configure consistent visualization style for the notebook"""
    colors = ["#A20025", "#6C8EBF"]  # Define a color palette
    sns.set_palette(sns.color_palette(colors))
    plt.rcParams['axes.titlesize'] = 18
    plt.rcParams['axes.labelsize'] = 16

setup_visualization_style()
%matplotlib inline


def setup_rdkit_drawing():
    """Configure RDKit drawing settings for consistent molecular visualizations"""
    d2d = Draw.MolDraw2DSVG(-1, -1)
    dopts = d2d.drawOptions()
    dopts.useBWAtomPalette()
    dopts.setHighlightColour((.635, .0, .145, .4))
    dopts.baseFontSize = 1.0
    dopts.additionalAtomLabelPadding = 0.15
    return dopts

rdkit_drawing_options = setup_rdkit_drawing()

### <b> <font color='#A20025'> Prepare Data Files

In [19]:
# Check if the file needs to be decompressed
import os
import subprocess

def prepare_data_files():
    """
    Prepare data files for the notebook by decompressing if needed.

    Returns:
        bool: True if files are ready
    """
    specs_sdf_path = "data/zinc_purchasable.sdf"
    specs_sdf_gz_path = "data/zinc_purchasable.sdf.gz"

    # Check if the decompressed file already exists
    if os.path.exists(specs_sdf_path):
        print(f"✓ File already exists: {specs_sdf_path}")
        return True

    # Check if the compressed file exists
    if not os.path.exists(specs_sdf_gz_path):
        print(f"✗ Compressed file not found: {specs_sdf_gz_path}")
        print("Please download the file from the repository or use the provided data.")
        return False

    # Decompress the file
    print(f"Decompressing {specs_sdf_gz_path}...")
    try:
        subprocess.run(['gzip', '-d', specs_sdf_gz_path], check=True)
        print(f"✓ Successfully decompressed file to {specs_sdf_path}")
        return True
    except subprocess.CalledProcessError:
        print("✗ Failed to decompress file using gzip")

        # Try with Python's gzip module as a fallback
        try:
            import gzip
            import shutil
            with gzip.open(specs_sdf_gz_path, 'rb') as f_in:
                with open(specs_sdf_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            print(f"✓ Successfully decompressed file to {specs_sdf_path} using Python")
            return True
        except Exception as e:
            print(f"✗ Failed to decompress file: {str(e)}")
            return False

# Prepare the data files
data_ready = prepare_data_files()
if not data_ready:
    print("\nPlease resolve the data issues before continuing.")

✓ File already exists: data/zinc_purchasable.sdf


In [16]:
!pip install rdkit joblib utils tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=750c33bbe546f0ecadaa7446589c694bb0ca59144a6b05685de666ae494c6855
  Stored in directory: /root/.cache/pip/wheels/b6/a1/81/1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


In [20]:
# Core data science packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Chemical informatics packages
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    Draw,
    PandasTools,
    FilterCatalog,
    Descriptors,
    MolFromSmiles,
    MolFromSmarts,
)
from rdkit import DataStructs
from rdkit.Chem.rdFingerprintGenerator import AdditionalOutput, GetMorganGenerator

# Utility functions
np.random.seed(42)

## <b> <font color='#A20025'> Loading a Virtual Screening Library

In [27]:
def load_sdf_file(file_path, smiles_name='smiles', mol_col_name='mol'):
    """
    Load an SDF file into a pandas DataFrame using RDKit.

    Parameters:
    -----------
    file_path : str
        Path to the SDF file
    smiles_name : str
        Name to use for the SMILES column
    mol_col_name : str
        Name to use for the molecule column

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing the molecules from the SDF
    """
    try:
        # Load SDF file
        print(f"Loading SDF file: {file_path}")
        df = PandasTools.LoadSDF(
            file_path,
            smilesName=smiles_name,
            molColName=None
        )

        # Keep only essential columns to reduce memory usage
        #essential_cols = ["PUBCHEM_SUBSTANCE_ID", smiles_name]
        #df = df[essential_cols]

        # Add molecule objects
        print("Adding RDKit molecule objects...")
        PandasTools.AddMoleculeColumnToFrame(df, smiles_name, mol_col_name)

        # Report basic statistics
        print(f"Loaded {len(df)} compounds")
        print(f"Valid molecules: {df[mol_col_name].notnull().sum()}")
        print(f"Invalid molecules: {df[mol_col_name].isnull().sum()}")

        return df

    except Exception as e:
        print(f"Error loading SDF file: {str(e)}")
        return None

# Load the Specs SDF file
specs = load_sdf_file("data/zinc_purchasable.sdf")

if specs is None:
    print("Failed to load the library. Please check the file path and format.")
else:
    # Show a sample of the data
    print("\nSample data:")
    display(specs.head(3))

Loading SDF file: data/zinc_purchasable.sdf
Adding RDKit molecule objects...
Error loading SDF file: 'smiles'
Failed to load the library. Please check the file path and format.


In [31]:
!ls -lrt /content/data/zinc_purchasable.sdf

-rw-r--r-- 1 root root 0 Feb 20 15:39 /content/data/zinc_purchasable.sdf
